In [2]:
import argparse, csv, sys, collections
import os
from tqdm import tqdm
import feather
import pandas as pd
from joblib import Parallel, delayed
import numpy as np

In [2]:
# from https://github.com/alexeygrigorev/outbrain-click-prediction-kaggle/blob/master/1_svm_data.py
def append_to_csv(batch, csv_file):
    props = dict(encoding='utf-8', index=False)
    if not os.path.exists(csv_file):
        batch.to_csv(csv_file, **props)
    else:
        batch.to_csv(csv_file, mode='a', header=False, **props)

def delete_file_if_exists(filename):
    if os.path.exists(filename):
        os.remove(filename)

def chunk_dataframe(df, n):
    for i in range(0, len(df), n):
        yield df.iloc[i:i+n]


In [3]:
def prepare_batch(batch):
    batch = batch.reset_index(drop=1)
    
    return batch

In [4]:
df_train  = feather.read_dataframe('../cache/train10_t.feather')

In [8]:
df_train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,...,mon,dow,doy,dcount,type,description,pd,wbe,wae,wfe
0,0,2013-01-01,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,...,1,1,1,734869,Holiday,Primer dia del ano,0,172,0,172
1,1,2013-01-01,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,...,1,1,1,734869,Holiday,Primer dia del ano,0,172,0,172
2,2,2013-01-01,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,...,1,1,1,734869,Holiday,Primer dia del ano,0,172,0,172
3,3,2013-01-01,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,...,1,1,1,734869,Holiday,Primer dia del ano,0,172,0,172
4,4,2013-01-01,25,108701,1.0,Unk,DELI,2644,1,Salinas,...,1,1,1,734869,Holiday,Primer dia del ano,0,172,0,172


In [9]:
df_train.shape

(125497040, 26)

In [10]:
df_train.columns

Index(['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion',
       'family', 'class', 'perishable', 'city', 'state', 'store_type',
       'cluster', 'dcoilwtico', 'transactions', 'dom', 'mon', 'dow', 'doy',
       'dcount', 'type', 'description', 'pd', 'wbe', 'wae', 'wfe'],
      dtype='object')

In [11]:
df_train = df_train.rename(columns = {'family':'item_family', 'class': 'item_class', 'type': 'h_type', 
                                      'description': 'h_desc'})

In [12]:
df_train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,item_family,item_class,perishable,city,...,mon,dow,doy,dcount,h_type,h_desc,pd,wbe,wae,wfe
0,0,2013-01-01,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,...,1,1,1,734869,Holiday,Primer dia del ano,0,172,0,172
1,1,2013-01-01,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,...,1,1,1,734869,Holiday,Primer dia del ano,0,172,0,172
2,2,2013-01-01,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,...,1,1,1,734869,Holiday,Primer dia del ano,0,172,0,172
3,3,2013-01-01,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,...,1,1,1,734869,Holiday,Primer dia del ano,0,172,0,172
4,4,2013-01-01,25,108701,1.0,Unk,DELI,2644,1,Salinas,...,1,1,1,734869,Holiday,Primer dia del ano,0,172,0,172


In [13]:
df_train.columns

Index(['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion',
       'item_family', 'item_class', 'perishable', 'city', 'state',
       'store_type', 'cluster', 'dcoilwtico', 'transactions', 'dom', 'mon',
       'dow', 'doy', 'dcount', 'h_type', 'h_desc', 'pd', 'wbe', 'wae', 'wfe'],
      dtype='object')

In [14]:
df_train['store_item_nbr'] = df_train['store_nbr'].astype('str') + '_' + df_train['item_nbr'].astype('str')

,id,store_nbr,item_nbr,unit_sales,onpromotion,item_family,item_class,perishable,city,state,...,dow,doy,dcount,h_type,h_desc,pd,wbe,wae,wfe,store_item_nbr
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,0,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,Santa Elena,...,1,1,734869,Holiday,Primer dia del ano,0,172,0,172,25_103665
2013-01-01,1,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,...,1,1,734869,Holiday,Primer dia del ano,0,172,0,172,25_105574
2013-01-01,2,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,...,1,1,734869,Holiday,Primer dia del ano,0,172,0,172,25_105575
2013-01-01,3,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,Santa Elena,...,1,1,734869,Holiday,Primer dia del ano,0,172,0,172,25_108079
2013-01-01,4,25,108701,1.0,Unk,DELI,2644,1,Salinas,Santa Elena,...,1,1,734869,Holiday,Primer dia del ano,0,172,0,172,25_108701


In [18]:
delete_file_if_exists('../cache/train10_t.csv')

for batch in tqdm(chunk_dataframe(df_train, n=1000000)):
    batch = prepare_batch(batch)
    append_to_csv(batch, '../cache/train10_t.csv')


126it [32:17, 15.38s/it]


In [19]:
f_cats = ['store_item_nbr', 'onpromotion', 'item_family', 'item_class', 'city', 'state', 'store_type', 
          'cluster', 'dom', 'dow', 'h_type', 'pd']

In [ ]:
counts = collections.defaultdict(lambda : [0, 0, 0])

In [31]:


def count_values_in_row(csv_path):
    for i, row in tqdm(enumerate(csv.DictReader(open(csv_path)), start=2)):
        label = float(row['unit_sales'])
        for j in f_cats:
            field = j
            value = row[field]
            if label <= 0.:
                counts[field+','+value][0] += 1
            else:
                counts[field+','+value][1] += 1
            counts[field+','+value][2] += 1
        if i % 1000000 == 0:
            sys.stderr.write('{0}m\n'.format(int(i/1000000)))

def count_test_values_in_row(csv_path):
    for i, row in tqdm(enumerate(csv.DictReader(open(csv_path)), start=2)):
#         label = float(row['unit_sales'])
        for j in f_cats:
            field = j
            value = row[field]
            counts[field+','+value][0] += 1
            counts[field+','+value][1] += 1
            counts[field+','+value][2] += 1
        if i % 1000000 == 0:
            sys.stderr.write('{0}m\n'.format(int(i/1000000)))
            
def print_results(dest_csv):
    fp = open(dest_csv, 'w')
    fp.write('Field,Value,Neg,Pos,Total,Ratio\n')
    for key, (neg, pos, total) in sorted(counts.items(), key=lambda x: x[1][2]):
        if total < 10:
            continue
        ratio = round(float(pos)/total, 5)
        counts_str = key+','+str(neg)+','+str(pos)+','+str(total)+','+str(ratio) + '\n'
        fp.write(counts_str)
    fp.close()

In [21]:
count_values_in_row('../cache/train10_t.csv')

999371it [00:20, 49280.87it/s]1m
1996026it [00:39, 50569.73it/s]2m
2995158it [00:59, 50420.09it/s]3m
3999234it [01:18, 50878.18it/s]4m
4996776it [01:37, 51082.62it/s]5m
5998459it [01:57, 51001.47it/s]6m
6994048it [02:16, 51204.91it/s]7m
7997855it [02:35, 51328.64it/s]8m
8994586it [02:54, 51449.76it/s]9m
9995279it [03:14, 51381.30it/s]10m
10994861it [03:34, 51320.29it/s]11m
11996048it [03:53, 51270.47it/s]12m
12999811it [04:13, 51297.49it/s]13m
13997856it [04:32, 51333.13it/s]14m
14997433it [04:51, 51463.81it/s]15m
15997544it [05:10, 51563.16it/s]16m
16999925it [05:29, 51559.38it/s]17m
17997829it [05:48, 51583.41it/s]18m
18999476it [06:08, 51588.21it/s]19m
19995094it [06:27, 51606.99it/s]20m
20996557it [06:46, 51696.46it/s]21m
21995853it [07:05, 51686.53it/s]22m
22998594it [07:24, 51797.95it/s]23m
23996092it [07:42, 51901.19it/s]24m
24996637it [08:00, 52011.17it/s]25m
25997354it [08:19, 51999.66it/s]26m
26998092it [08:39, 51994.36it/s]27m
27998465it [08:58, 52013.15it/s]28m
28999853it [

In [23]:
print_results('../cache/counts_train10_t.csv')

In [3]:
df_test  = feather.read_dataframe('../cache/test7_t.feather')

In [13]:
df_test = df_test.rename(columns = {'family':'item_family', 'class': 'item_class', 'type': 'h_type', 
                                      'description': 'h_desc'})

In [14]:
df_test['store_item_nbr'] = df_test['store_nbr'].astype('str') + '_' + df_test['item_nbr'].astype('str')

In [35]:
delete_file_if_exists('../cache/test7_t.csv')

for batch in tqdm(chunk_dataframe(df_test, n=1000000)):
    batch = prepare_batch(batch)
    append_to_csv(batch, '../cache/test7_t.csv')


4it [00:49, 12.27s/it]


In [36]:
counts = collections.defaultdict(lambda : [0, 0, 0])

In [37]:
count_test_values_in_row('../cache/test7_t.csv')

998410it [00:23, 41832.78it/s]1m
1996759it [00:45, 44037.44it/s]2m
2996361it [01:06, 45201.35it/s]3m
3370464it [01:14, 45236.00it/s]


In [38]:
print_results('../cache/counts_test7_t.csv')

In [15]:
df = pd.read_csv('../cache/counts_train10_t.csv')

In [16]:
df.head()

,Field,Value,Neg,Pos,Total,Ratio
0,store_item_nbr,41_2061033,0,10,10,1.0
1,store_item_nbr,39_1463887,0,10,10,1.0
2,store_item_nbr,33_1950263,0,10,10,1.0
3,store_item_nbr,4_2036465,0,10,10,1.0
4,store_item_nbr,35_1463859,0,10,10,1.0


In [17]:
df.shape

(171613, 6)

In [18]:
df1 = df[df.Total >= 12000000]

In [19]:
df1.shape


(26, 6)

In [20]:
df1

,Field,Value,Neg,Pos,Total,Ratio
171587,cluster,10,562,12232875,12233437,0.99995
171588,cluster,14,706,13239680,13240386,0.99995
171589,cluster,6,526,13318408,13318934,0.99996
171590,city,Guayaquil,1021,16709655,16710676,0.99994
171591,item_family,CLEANING,1026,17014574,17015600,0.99994
171592,dow,3,1412,17112396,17113808,0.99992
171593,dow,4,1149,17507133,17508282,0.99993
171594,dow,2,1198,17520082,17521280,0.99993
171595,item_family,BEVERAGES,1361,17566027,17567388,0.99992
171596,dow,1,1327,17663995,17665322,0.99992


In [48]:
df1.to_csv('../cache/counts_dense_train10_t.csv')

In [21]:
df_train['dense_count'] = 0

In [36]:
df_train = df_train.drop('dense_count', axis=1)

In [ ]:
df_train = df_train.set_index('date')

In [17]:
df_train.head()

,id,store_nbr,item_nbr,unit_sales,onpromotion,item_family,item_class,perishable,city,state,...,dow,doy,dcount,h_type,h_desc,pd,wbe,wae,wfe,store_item_nbr
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,0,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,Santa Elena,...,1,1,734869,Holiday,Primer dia del ano,0,172,0,172,25_103665
2013-01-01,1,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,...,1,1,734869,Holiday,Primer dia del ano,0,172,0,172,25_105574
2013-01-01,2,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,...,1,1,734869,Holiday,Primer dia del ano,0,172,0,172,25_105575
2013-01-01,3,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,Santa Elena,...,1,1,734869,Holiday,Primer dia del ano,0,172,0,172,25_108079
2013-01-01,4,25,108701,1.0,Unk,DELI,2644,1,Salinas,Santa Elena,...,1,1,734869,Holiday,Primer dia del ano,0,172,0,172,25_108701


In [18]:
df_train.shape

(125497040, 26)

In [19]:
df_train = df_train.reset_index()

In [20]:
df_train.shape

(125497040, 27)

In [21]:
df_train.head()

,date,id,store_nbr,item_nbr,unit_sales,onpromotion,item_family,item_class,perishable,city,...,dow,doy,dcount,h_type,h_desc,pd,wbe,wae,wfe,store_item_nbr
0,2013-01-01,0,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,...,1,1,734869,Holiday,Primer dia del ano,0,172,0,172,25_103665
1,2013-01-01,1,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,...,1,1,734869,Holiday,Primer dia del ano,0,172,0,172,25_105574
2,2013-01-01,2,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,...,1,1,734869,Holiday,Primer dia del ano,0,172,0,172,25_105575
3,2013-01-01,3,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,...,1,1,734869,Holiday,Primer dia del ano,0,172,0,172,25_108079
4,2013-01-01,4,25,108701,1.0,Unk,DELI,2644,1,Salinas,...,1,1,734869,Holiday,Primer dia del ano,0,172,0,172,25_108701


In [25]:
past_date = np.datetime64('2015-01-01')

df_train = df_train[(df_train.date.apply(np.datetime64) >= past_date)]

In [26]:
df_train.shape

(86902776, 27)

In [27]:
df_train.head()

,date,id,store_nbr,item_nbr,unit_sales,onpromotion,item_family,item_class,perishable,city,...,dow,doy,dcount,h_type,h_desc,pd,wbe,wae,wfe,store_item_nbr
38594264,2015-01-01,38594264,25,103665,12.0,False,BREAD/BAKERY,2712,1,Salinas,...,3,1,735599,Holiday,Primer dia del ano,0,68,0,68,25_103665
38594265,2015-01-01,38594265,25,105575,23.0,False,GROCERY I,1045,0,Salinas,...,3,1,735599,Holiday,Primer dia del ano,0,68,0,68,25_105575
38594266,2015-01-01,38594266,25,108634,1.0,False,GROCERY I,1075,0,Salinas,...,3,1,735599,Holiday,Primer dia del ano,0,68,0,68,25_108634
38594267,2015-01-01,38594267,25,108698,6.0,False,DELI,2644,1,Salinas,...,3,1,735599,Holiday,Primer dia del ano,0,68,0,68,25_108698
38594268,2015-01-01,38594268,25,108786,6.0,False,CLEANING,3044,0,Salinas,...,3,1,735599,Holiday,Primer dia del ano,0,68,0,68,25_108786


In [28]:
df_train.tail()

,date,id,store_nbr,item_nbr,unit_sales,onpromotion,item_family,item_class,perishable,city,...,dow,doy,dcount,h_type,h_desc,pd,wbe,wae,wfe,store_item_nbr
125497035,2017-08-15,125497035,54,2089339,4.0,False,GROCERY I,1006,0,El Carmen,...,1,227,736556,Work Day,Normal,1,0,70,70,54_2089339
125497036,2017-08-15,125497036,54,2106464,1.0,True,BEVERAGES,1148,0,El Carmen,...,1,227,736556,Work Day,Normal,1,0,70,70,54_2106464
125497037,2017-08-15,125497037,54,2110456,192.0,False,BEVERAGES,1120,0,El Carmen,...,1,227,736556,Work Day,Normal,1,0,70,70,54_2110456
125497038,2017-08-15,125497038,54,2113914,198.0,True,CLEANING,3040,0,El Carmen,...,1,227,736556,Work Day,Normal,1,0,70,70,54_2113914
125497039,2017-08-15,125497039,54,2116416,2.0,False,GROCERY I,1060,0,El Carmen,...,1,227,736556,Work Day,Normal,1,0,70,70,54_2116416


In [29]:
import os
x = {i for i in range(20)}
os.sched_setaffinity(0, x)

In [30]:
# now split in to various stores

In [31]:
len(set(df_train.store_nbr))

54

In [32]:
set(range(1,55)) - set(df_train.store_nbr)

set()

In [34]:
set(df_train.store_nbr) - set(range(1,55))

set()

In [35]:
import gc
gc.enable()

In [36]:
for i in tqdm(range(1,55)):
    df = df_train[df_train.store_nbr == i]
    fn = '../cache/train10_t_store{}.csv'.format(i)
    df.to_csv(fn, index=False)
    del df

100%|██████████| 54/54 [22:27<00:00, 24.95s/it]


In [37]:
gc.collect()

449

In [ ]:
set(range(1,55)) - set(df_test.store_nbr)

In [ ]:
len(set(df_test.store_nbr))

In [39]:
df_test.shape

(3370464, 26)

In [40]:
df_train.shape

(125497040, 27)

In [42]:
for i in tqdm(range(1,55)):
    df = df_test[df_test.store_nbr == i]
    fn = '../cache/test7_t_store{}.csv'.format(i)
    df.to_csv(fn, index=False)
    del df


 15%|█▍        | 8/54 [00:08<00:51,  1.11s/it]
Exception in thread Thread-8:
Traceback (most recent call last):
  File "/home/watts/anaconda3/envs/rss/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/watts/anaconda3/envs/rss/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 54/54 [00:48<00:00,  1.11it/s]


In [43]:
gc.collect()

424

In [3]:
df_test  = pd.read_csv('../cache/test7_t.csv')

In [4]:
df_test.T

,0,1,2,3,4,5,6,7,8,9,...,3370454,3370455,3370456,3370457,3370458,3370459,3370460,3370461,3370462,3370463
id,125497040,125497041,125497042,125497043,125497044,125497045,125497046,125497047,125497048,125497049,...,128867494,128867495,128867496,128867497,128867498,128867499,128867500,128867501,128867502,128867503
date,2017-08-16,2017-08-16,2017-08-16,2017-08-16,2017-08-16,2017-08-16,2017-08-16,2017-08-16,2017-08-16,2017-08-16,...,2017-08-31,2017-08-31,2017-08-31,2017-08-31,2017-08-31,2017-08-31,2017-08-31,2017-08-31,2017-08-31,2017-08-31
store_nbr,1,1,1,1,1,1,1,1,1,1,...,54,54,54,54,54,54,54,54,54,54
item_nbr,96995,99197,103501,103520,103665,105574,105575,105576,105577,105693,...,2130526,2130553,2131010,2131572,2131699,2132163,2132318,2132945,2132957,2134244
onpromotion,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
item_family,GROCERY I,GROCERY I,CLEANING,GROCERY I,BREAD/BAKERY,GROCERY I,GROCERY I,GROCERY I,GROCERY I,GROCERY I,...,GROCERY I,"LIQUOR,WINE,BEER","LIQUOR,WINE,BEER",GROCERY I,GROCERY I,GROCERY I,GROCERY I,GROCERY I,GROCERY I,"LIQUOR,WINE,BEER"
item_class,1093,1067,3008,1028,2712,1045,1045,1045,1045,1034,...,1030,1318,1328,1002,1002,1040,1002,1026,1068,1364
perishable,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
city,Quito,Quito,Quito,Quito,Quito,Quito,Quito,Quito,Quito,Quito,...,El Carmen,El Carmen,El Carmen,El Carmen,El Carmen,El Carmen,El Carmen,El Carmen,El Carmen,El Carmen
state,Pichincha,Pichincha,Pichincha,Pichincha,Pichincha,Pichincha,Pichincha,Pichincha,Pichincha,Pichincha,...,Manabi,Manabi,Manabi,Manabi,Manabi,Manabi,Manabi,Manabi,Manabi,Manabi


In [5]:
df_sub = pd.read_csv('../submissions/lgb.csv')

In [6]:
df_sub.head()

,id,unit_sales
0,125497040,0.2087
1,125497041,0.3790
2,125497042,0.0000
3,125497043,1.4433
4,125497044,1.9638


In [7]:
df_test = pd.merge(df_test, df_sub, how='left', on='id')

In [8]:
df_test.isnull().sum()

id                0
date              0
store_nbr         0
item_nbr          0
onpromotion       0
item_family       0
item_class        0
perishable        0
city              0
state             0
store_type        0
cluster           0
dcoilwtico        0
transactions      0
dom               0
mon               0
dow               0
doy               0
dcount            0
h_type            0
h_desc            0
pd                0
wbe               0
wae               0
wfe               0
store_item_nbr    0
unit_sales        0
dtype: int64

In [10]:
df_test.shape

(3370464, 27)

In [9]:
for i in tqdm(range(1,55)):
    df = df_test[df_test.store_nbr == i]
    fn = '../cache/bkup2/test7_1_t_store{}.csv'.format(i)
    df.to_csv(fn, index=False, float_format='%.5f')
    del df

100%|██████████| 54/54 [00:52<00:00,  1.03it/s]
